# Case Study- Housing Prices

In [1]:
# Pandas is for using data structures
import pandas as pd
# statsmodels contain modules for regression and time series analysis
import statsmodels.api as sm
# numpy is for numerical computing of array and matrix
import numpy as np
# Matplotlib is a plotting package
import matplotlib.pyplot as plt
# matplotlib Showing the plot right after the current code  
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# basic statistics package
import scipy.stats as stats
# Calculate VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [2]:
# functions from last lab
def four_in_one(dataframe,model):
    fitted_y = model.fittedvalues
    studentized_residuals = model.get_influence().resid_studentized_internal
    plt.figure(figsize=(10,10))
    ax1 = plt.subplot(221)
    QQ=stats.probplot(studentized_residuals, dist="norm", plot=plt)
    ax1.set_title('Normal Q-Q')
    ax1.set_xlabel('Normal Quantiles')
    ax1.set_ylabel('Studentized Residuals');
    
    # annotations for the top 3 residulas (if their absolute values are larger than 2)
    stdRes_id = np.flip(np.argsort(np.abs(studentized_residuals)), 0)
    if abs(stdRes_id[0])>2:
        stdRes_top_3_id = stdRes_id[:3]
        abs_norm_resid = np.flip(np.argsort(np.abs(QQ[0][1])), 0)
        abs_norm_resid_top_3 = abs_norm_resid[:3]
        for r, i in enumerate(abs_norm_resid_top_3):
            ax1.annotate(stdRes_top_3_id[r],
                                       xy=(QQ[0][0][i],
                                           QQ[0][1][i]));

    ax2 = plt.subplot(222)
    ax2.hist(studentized_residuals)
    ax2.set_xlabel('Studentized Residuals')
    ax2.set_ylabel('Count')
    ax2.set_title('Histogram')

    ax3 = plt.subplot(223)
    t = range(dataframe.shape[0])
    ax3.scatter(t, studentized_residuals)
    ax3.set_xlabel('Observation order')
    ax3.set_ylabel('Studentized Residuals')
    ax3.set_title('Time series plot of studentized residuals')

    ax4 = plt.subplot(224)
    ax4 = sns.residplot(fitted_y, studentized_residuals,
                              lowess=True,
                              scatter_kws={'alpha': 0.5},
                              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    ax4.set_title('Studentized Residuals vs Fitted values')
    ax4.set_xlabel('Fitted values')
    ax4.set_ylabel('Studentized Residuals');
    
def getvif(X):
    X = sm.add_constant(X)
    vif = pd.DataFrame()
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["Predictors"] = X.columns
    return(vif.drop(index = 0).round(2)) 

def prediction(new_X,model):
    return(model.get_prediction(new_X).summary_frame())

## Loading data

In [3]:
# Impport data
df = pd.read_excel('housingprices.xlsx')
N = df.shape

## Exploratory Data Analysis

In [4]:
#Step 1: Exploratory Data Analysis
#Summary statistics
print(df.describe())

            Price       Beds      Baths  Square Feet  Miles to Resort  \
count   45.000000  45.000000  45.000000    45.000000        45.000000   
mean   391.191111   3.288889   2.333333  1900.711111        12.400000   
std    132.600555   1.160373   0.992872   683.044708        13.964891   
min    160.000000   1.000000   1.000000   768.000000         0.000000   
25%    300.000000   3.000000   2.000000  1440.000000         2.000000   
50%    378.000000   3.000000   2.000000  1858.000000         7.000000   
75%    496.000000   4.000000   2.500000  2160.000000        20.000000   
max    690.000000   6.000000   4.750000  3875.000000        52.000000   

       Miles to Base      Acres       Cars  Years Old         DoM  
count      45.000000  45.000000  45.000000  45.000000   45.000000  
mean       13.911111   2.309111   1.422222  22.133333  122.022222  
std        14.172352   6.778599   1.033284  20.304052   91.611951  
min         1.000000   0.100000   0.000000   3.000000   16.000000  
25

In [5]:
df.corr()

,Price,Beds,Baths,Square Feet,Miles to Resort,Miles to Base,Acres,Cars,Years Old,DoM
Price,1.000000,0.675309,0.800149,0.697042,-0.539094,-0.633233,0.025074,0.452337,-0.355093,0.229842
Beds,0.675309,1.000000,0.733179,0.728219,-0.350913,-0.424059,-0.147327,0.104465,-0.340262,-0.097125
Baths,0.800149,0.733179,1.000000,0.790132,-0.374544,-0.488044,-0.193023,0.435677,-0.326659,0.120538
Square Feet,0.697042,0.728219,0.790132,1.000000,-0.189463,-0.297245,-0.145631,0.372782,-0.303699,-0.010989
Miles to Resort,-0.539094,-0.350913,-0.374544,-0.189463,1.000000,0.948018,0.295799,-0.158448,0.108176,-0.221870
Miles to Base,-0.633233,-0.424059,-0.488044,-0.297245,0.948018,1.000000,0.263350,-0.261215,0.221110,-0.184077
Acres,0.025074,-0.147327,-0.193023,-0.145631,0.295799,0.263350,1.000000,0.147369,-0.029493,0.328835
Cars,0.452337,0.104465,0.435677,0.372782,-0.158448,-0.261215,0.147369,1.000000,-0.271400,0.313697
Years Old,-0.355093,-0.340262,-0.326659,-0.303699,0.108176,0.221110,-0.029493,-0.271400,1.000000,0.007745
DoM,0.229842,-0.097125,0.120538,-0.010989,-0.221870,-0.184077,0.328835,0.313697,0.007745,1.000000


# Size Related

In [6]:

X = df.loc[:,['Beds','Baths','Square Feet']]
Y = df['Price']

model_fit = sm.OLS(Y,sm.add_constant(X['Beds'])).fit()
display(model_fit.summary())

model_fit = sm.OLS(Y,sm.add_constant(X['Baths'])).fit()
display(model_fit.summary())

model_fit = sm.OLS(Y,sm.add_constant(X['Square Feet'])).fit()
display(model_fit.summary())



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.456
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     36.05
Date:                Fri, 21 Jan 2022   Prob (F-statistic):           3.61e-07
Time:                        11:52:01   Log-Likelihood:                -269.58
No. Observations:                  45   AIC:                             543.2
Df Residuals:                      43   BIC:                             546.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        137.3863     44.770      3.069      0.004      47.099     227.673
Beds          77.1704     12.853      6.004      0.000      51.250     103.090
==============================================================================
Omnibus:                        6.992   Durbin-Watson:                   1.721
Prob(Omnibus):                  0.030   Jarque-Bera (JB):                6.876
Skew:                           0.564   Prob(JB):                       0.0321
Kurtosis:                       4.547   Cond. No.                         11.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     76.52
Date:                Fri, 21 Jan 2022   Prob (F-statistic):           4.25e-11
Time:                        11:52:01   Log-Likelihood:                -260.27
No. Observations:                  45   AIC:                             524.5
Df Residuals:                      43   BIC:                             528.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        141.8468     30.924      4.587      0.000      79.482     204.211
Baths        106.8619     12.216      8.748      0.000      82.226     131.498
==============================================================================
Omnibus:                       22.127   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.905
Skew:                           1.444   Prob(JB):                     9.69e-09
Kurtosis:                       6.367   Cond. No.                         7.41
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.486
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     40.64
Date:                Fri, 21 Jan 2022   Prob (F-statistic):           1.04e-07
Time:                        11:52:01   Log-Likelihood:                -268.31
No. Observations:                  45   AIC:                             540.6
Df Residuals:                      43   BIC:                             544.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         133.9909     42.819      3.129      0.003      47.638     220.344
Square Feet     0.1353      0.021      6.375      0.000       0.093       0.178
==============================================================================
Omnibus:                        4.771   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.092   Jarque-Bera (JB):                5.549
Skew:                           0.070   Prob(JB):                       0.0624
Kurtosis:                       4.714   Cond. No.                     6.02e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
model_fit = sm.OLS(Y,sm.add_constant(X)).fit()
display(model_fit.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.636
Method:                 Least Squares   F-statistic:                     26.68
Date:                Fri, 21 Jan 2022   Prob (F-statistic):           9.86e-10
Time:                        11:52:01   Log-Likelihood:                -258.92
No. Observations:                  45   AIC:                             525.8
Df Residuals:                      41   BIC:                             533.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         105.2611     38.410      2.740      0.009      27.690     182.832
Beds           17.8354     16.354      1.091      0.282     -15.191      50.862
Baths          79.9010     21.370      3.739      0.001      36.743     123.059
Square Feet     0.0215      0.031      0.697      0.490      -0.041       0.084
==============================================================================
Omnibus:                       24.438   Durbin-Watson:                   1.937
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.854
Skew:                           1.513   Prob(JB):                     6.70e-11
Kurtosis:                       6.980   Cond. No.                     6.61e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.61e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:

corrMatrix=pd.concat([X,Y],axis=1).corr()
corrMatrix.style.background_gradient(cmap='coolwarm')


,Beds,Baths,Square Feet,Price
Beds,1.000000,0.733179,0.728219,0.675309
Baths,0.733179,1.000000,0.790132,0.800149
Square Feet,0.728219,0.790132,1.000000,0.697042
Price,0.675309,0.800149,0.697042,1.000000


In [9]:
sns.pairplot(corrMatrix)

NameError: name 'sns' is not defined

# Location Related

In [ ]:

X = df.loc[:,['Miles to Resort','Miles to Base']]
Y = df['Price']

model_fit = sm.OLS(Y,sm.add_constant(X['Miles to Resort'])).fit()
display(model_fit.summary())

model_fit = sm.OLS(Y,sm.add_constant(X['Miles to Base'])).fit()
display(model_fit.summary())


In [ ]:
model_fit = sm.OLS(Y,sm.add_constant(X)).fit()
display(model_fit.summary())

In [ ]:
corrMatrix=pd.concat([X,Y],axis=1).corr()
corrMatrix.style.background_gradient(cmap='coolwarm')

## Full Model

In [ ]:
X = df.drop(columns='Price')
Y = df['Price']


model_fit = sm.OLS(Y,sm.add_constant(X)).fit()
display(model_fit.summary())

In [ ]:
#VIF
X = df.drop(columns='Price')
getvif(X)

## Final Model

In [ ]:
X = df.loc[:,['Baths','Square Feet','Miles to Resort','Acres']]
Y = df['Price']


model_fit = sm.OLS(Y,sm.add_constant(X)).fit()
display(model_fit.summary())

In [ ]:
# residuals plots
four_in_one(df,model_fit)